# RNN hihello

#입력 5
#sequence =6
#hiddent =5
#batch =1 => 문자열의 개수


In [1]:
import tensorflow as tf
import numpy as np

In [2]:
tf.set_random_seed(32147354)

In [24]:
idx2char = ['h', 'i', 'e', 'l', 'o']

In [6]:
x_data = [[0,1,0,2,3,3]] # hihell


In [7]:
x_one_hot = [[
    [1,0,0,0,0], #h 0
    [0,1,0,0,0], #i 1
    [1,0,0,0,0], #h 0
    [0,0,1,0,0], #e 2
    [0,0,0,1,0], #l 3
    [0,0,0,3,0], #l 3
]]

In [8]:
y_data = [[1,0,2,3,3,4]] #ihello

In [9]:
num_classes = 5
input_dim =5 # one hot size
hidden_size = 5 #output for the LSTM. 5 to directly predict one-hot
batch_size = 1 #one sentence / 건내줄 데이터의 수라고 생각하면 된다.
sequence_length = 6 # ihello , 
                    #셀을 몇개를 펼칠것인지 ? 입력데이터의 모양에 따라 결졍된다.
learning_rate = 0.1

In [10]:
X = tf.placeholder(tf.float32,[None,sequence_length,input_dim]) 
#[batchszie, sequnce_lenght, input data ]
# X-onehot

In [11]:
Y = tf.placeholder(tf.int32,[None,sequence_length])

In [12]:
cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)

In [13]:
initial_state = cell.zero_state(batch_size,tf.float32)

In [14]:
outputs, _states = tf.nn.dynamic_rnn(
    cell, X, initial_state=initial_state, dtype=tf.float32)

In [15]:
# FC layer
X_for_fc = tf.reshape(outputs, [-1, hidden_size])
# fc_w = tf.get_variable("fc_w", [hidden_size, num_classes])
# fc_b = tf.get_variable("fc_b", [num_classes])
# outputs = tf.matmul(X_for_fc, fc_w) + fc_b

In [16]:
outputs = tf.contrib.layers.fully_connected(
    inputs=X_for_fc, num_outputs=num_classes, activation_fn=None)

In [17]:
# reshape out for sequence_loss
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])

In [18]:
weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss) #학습

In [19]:
prediction = tf.argmax(outputs, axis=2)

In [20]:
prediction

<tf.Tensor 'ArgMax:0' shape=(1, 6) dtype=int64>

In [25]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(50):
        l, _ = sess.run([loss, train], feed_dict={X: x_one_hot, Y: y_data})
        result = sess.run(prediction, feed_dict={X: x_one_hot})
        print(i, "loss:", l, "prediction: ", result, "true Y: ", y_data)

        # print char using dic
        result_str = [idx2char[c] for c in np.squeeze(result)]
        print("\tPrediction str: ", ''.join(result_str))

0 loss: 1.56013 prediction:  [[3 3 3 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  lllllo
1 loss: 1.41796 prediction:  [[3 3 3 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  lllllo
2 loss: 1.31088 prediction:  [[3 3 3 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  lllllo
3 loss: 1.19411 prediction:  [[3 3 3 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  lllllo
4 loss: 1.05712 prediction:  [[2 3 3 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  ellllo
5 loss: 0.911059 prediction:  [[1 2 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  ieello
6 loss: 0.779269 prediction:  [[1 2 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  ieello
7 loss: 0.65474 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  ihello
8 loss: 0.53567 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  ihello
9 loss: 0.432158 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  ihel